<a href="https://colab.research.google.com/github/ishleenkaur256/pfa_project/blob/main/Copy_of_PFA_Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import bs4
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import ticker
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
import operator
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image
from sklearn.neighbors import KNeighborsClassifier
import numpy
import random
seed = 42
numpy.random.seed(42)

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

from sklearn import  svm

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from urllib.parse import urljoin
import random


CATEGORIES = {
    "NATIONAL": "https://www.thehindu.com/news/national/",
    "BUSINESS": "https://www.thehindu.com/business/",
    "LIFE-STYLE": "https://www.thehindu.com/life-and-style/",
    "TECHNOLOGY": "https://www.thehindu.com/sci-tech/technology/",
    "EDUCATION": "https://www.thehindu.com/education/",
}
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
OUTPUT_FILENAME = '/content/drive/MyDrive/PFA_Final_Project/thehindu_dataset.csv'
scraped_articles = []

BASE_DELAY = 1.0
MAX_CONTAINERS_PER_PAGE = 50
PAGES_TO_SCRAPE = 10


def get_summary_from_article_page(article_link, headers):
    """Fetches the full article page and returns the first paragraph (the summary)."""
    summary = "Summary extraction failed"
    try:
        deep_response = requests.get(article_link, headers=headers, timeout=10)
        deep_response.raise_for_status()
        deep_soup = BeautifulSoup(deep_response.text, 'html.parser')

        # Target the main content wrapper
        article_body_wrapper = deep_soup.find('div', class_=re.compile(r'article-body|content-body|body-text|main-article-content|story-text'))

        if article_body_wrapper:
            first_paragraph = article_body_wrapper.find('p')
            if first_paragraph:
                summary = first_paragraph.get_text(strip=True)[:200] + "..."
                return summary

        fallback_paragraph = deep_soup.find('div', id=re.compile(r'content')).find('p')
        if fallback_paragraph:
             summary = fallback_paragraph.get_text(strip=True)[:200] + "..."
             return summary

    except requests.exceptions.RequestException:
        pass
    except Exception:
        pass

    return summary


print(f"--- Starting Scraping: {PAGES_TO_SCRAPE} Pages x {len(CATEGORIES)} Categories ---")
total_new_articles = 0

for category_name, base_url in CATEGORIES.items():

    # Print only once per category loop
    print(f"\n--- Processing Category: {category_name} (Pages 1 to {PAGES_TO_SCRAPE}) ---")

    for page_num in range(1, PAGES_TO_SCRAPE + 1):

        # 1. Construct the URL
        if page_num == 1:
            category_url = base_url
        else:
            category_url = f"{base_url}?page={page_num}"

        try:
            # 2. Fetch the page
            response = requests.get(category_url, headers=HEADERS, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # 3. Extract article containers
            article_containers = soup.find_all('div', class_=re.compile(r'article-card|story-card|list-item'))
            if not article_containers:
                article_containers = soup.find_all(['h3', 'h2'], class_=re.compile(r''))
                article_containers = [tag for tag in article_containers if tag.find('a')]

            articles_processed_in_page = 0

            # 4. Deep Dive Loop
            for container in article_containers:
                if articles_processed_in_page >= MAX_CONTAINERS_PER_PAGE:
                    break

                link_tag = container.find('a', href=True)
                if not link_tag: continue

                title = link_tag.get_text(strip=True)
                link = urljoin(category_url, link_tag['href'])

                # Filtering and Duplicate Check
                if not title or len(title) < 10 or 'subscription' in link.lower() or 'opinion' in link.lower(): continue
                if any(article['Link'] == link for article in scraped_articles): continue # Skip duplicates

                # Execute Deep Dive
                delay_time = BASE_DELAY + random.uniform(0.1, 0.5)
                time.sleep(delay_time)

                summary = get_summary_from_article_page(link, HEADERS)

                scraped_articles.append({
                    'Title': title,
                    'Summary': summary,
                    'Category': category_name,
                    'Link': link
                })

                articles_processed_in_page += 1
                total_new_articles += 1

            # Print minimal status update only if articles were added
            if articles_processed_in_page > 0:
                print(f"  > Successfully added {articles_processed_in_page} articles from Page {page_num}.")

        except requests.exceptions.RequestException as e:
            print(f"FAILED to fetch {category_name} Page {page_num}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred in {category_name} Page {page_num}: {e}")

# --- Final Output and CSV Saving ---

print("\n" + "="*70)
print(f"Scraping Complete! Total unique articles collected: {len(scraped_articles)}")
print("="*70)

df = pd.DataFrame(scraped_articles)
df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')

print(f"Data saved to **{OUTPUT_FILENAME}**")

if not df.empty:
    print(f"Sample of final data (first 5 rows):\n{df.head().to_markdown(index=False)}")
else:
    print("No articles were successfully extracted.")

In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/PFA_Final_Project/thehindu_dataset.csv")

In [ ]:
df

In [ ]:
df["text"] = df["Title"].astype(str) + " " + df["Summary"].astype(str)


In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["label"] = le.fit_transform(df["Category"])


In [ ]:
df

In [ ]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)

dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]


In [ ]:
train_dataset

In [ ]:
cols_to_remove = ["Title", "Summary", "Category", "Link"]
train_dataset = train_dataset.remove_columns(cols_to_remove)
test_dataset   = test_dataset.remove_columns(cols_to_remove)


In [ ]:
train_dataset.column_names


In [ ]:
test_dataset.column_names

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "albert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

num_labels = df["label"].nunique()

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)


In [ ]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)


In [ ]:
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

test_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/PFA_Final_Project/albert-news-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=20,
    load_best_model_at_end=True,
    report_to="none"
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)


In [ ]:
trainer.train()


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

predictions = trainer.predict(test_dataset)

preds = predictions.predictions.argmax(axis=-1)
true = predictions.label_ids

print("Accuracy:", accuracy_score(true, preds))

print("\nClassification Report:\n")
print(classification_report(true, preds, target_names=le.classes_))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(true, preds)

plt.figure(figsize=(7,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_,
            yticklabels=le.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import torch

def predict_news_category(title, summary):
    # Combine training dataset
    text = title + " " + summary

    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Put model in evaluation mode
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class ID
    predicted_label_id = torch.argmax(outputs.logits, dim=1).item()

    # Convert number back to label name
    predicted_label = le.inverse_transform([predicted_label_id])[0]

    return predicted_label


In [ ]:
predict_news_category(
    "Sensex rises 350 points amid global market optimism",
    "Investors reacted positively to corporate earnings and global cues."
)


In [ ]:
predict_news_category(
    "ISRO successfully tests new space propulsion system",
    "The breakthrough could enhance satellite maneuvering and deep-space missions."
)


In [ ]:
model.save_pretrained("/content/drive/MyDrive/PFA_Final_Project/news_classifier_model")
tokenizer.save_pretrained("/content/drive/MyDrive/PFA_Final_Project/news_classifier_model")


In [ ]:
import pickle

with open("/content/drive/MyDrive/PFA_Final_Project/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


Gradio

In [ ]:
!pip install gradio


In [ ]:
import gradio as gr


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pickle

# Load saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/PFA_Final_Project/news_classifier_model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/PFA_Final_Project/news_classifier_model")

# Load label encoder
with open("/content/drive/MyDrive/PFA_Final_Project/label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

model.eval()


In [ ]:
def classify_news(title, summary):
    text = title + " " + summary

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)

    pred_id = torch.argmax(outputs.logits, dim=1).item()
    label = le.inverse_transform([pred_id])[0]

    return label


In [ ]:
import pandas as pd

def classify_csv(file):
    df = pd.read_csv(file.name)

    # Check required columns
    if not {"Title", "Summary"} <= set(df.columns):
        return "CSV must contain 'Title' and 'Summary' columns.", None

    predictions = []

    for i in range(len(df)):
        title = str(df.iloc[i]["Title"])
        summary = str(df.iloc[i]["Summary"])

        pred = classify_news(title, summary)
        predictions.append(pred)

    df["Predicted_Category"] = predictions

    # Save output CSV
    output_path = "/content/drive/MyDrive/PFA_Final_Project/predicted_results.csv"
    df.to_csv(output_path, index=False)

    return df, output_path


In [ ]:
custom_css = custom_css = """
/* Background Image */
.gradio-container {
    background: url('https://images.unsplash.com/photo-1507842217343-583bb7270b66?auto=format&fit=crop&w=1350&q=80') no-repeat center center fixed ;
    background-size: cover !important;
    font-family: 'Poppins', sans-serif;
    position: relative;
    z-index: 1;
}

.gradio-container::before {
    content: "";
    position: absolute;
    top: 0;
    left: 0;
    height: 100%;
    width: 100%;
    background: rgba(0,0,0,0.55);
    z-index: 1;
    pointer-events: none;
}


.gradio-container * {
    position: relative;
    z-index: 3;
    color: #4e2a0a !important;
}


.prose.svelte-ydeks8 h1,
h1.svelte-ydeks8,
.prose h1 {
    color: #ffffff ;
    font-size: 48px ;
    font-weight: 800 ;
    text-align: center ;

    text-shadow:
        0 0 10px rgba(255, 220, 160, 0.9),
        0 0 20px rgba(255, 180, 90, 0.8),
        0 0 40px rgba(255, 150, 50, 0.7),
        0 0 60px rgba(255, 120, 30, 0.6);

    animation: headingGlow 2.5s infinite alternate ease-in-out;
}

@keyframes headingGlow {
    from {
        text-shadow:
            0 0 6px rgba(255, 220, 160, 0.7),
            0 0 14px rgba(255, 180, 90, 0.6),
            0 0 25px rgba(255, 150, 50, 0.4);
    }
    to {
        text-shadow:
            0 0 12px rgba(255, 240, 200, 1),
            0 0 28px rgba(255, 200, 120, 0.9),
            0 0 45px rgba(255, 160, 80, 0.7),
            0 0 70px rgba(255, 140, 60, 0.6);
    }
}



.typing {
    width: 100%;
    border-right: 3px solid white;
    white-space: nowrap;
    overflow: hidden;
    animation: typing 4s steps(30), blink 0.75s step-end infinite;
}
@keyframes typing {
    from { width: 0 }
    to   { width: 100% }
}
@keyframes blink {
    from, to { border-color: transparent }
    50% { border-color: white }
}



button {
    background: linear-gradient(90deg, #b38752, #84592c) ;
    color: #fff7e6 ;
    font-weight: bold ;
    border-radius: 12px ;
    padding: 10px 20px ;
    font-size: 17px ;
}
button:hover {
    transform: scale(1.05);
    background: linear-gradient(90deg, #c99a60, #966736) ;
}

label, .gr-label, .form label {
    color: #e6cba8 ;
    font-weight: 700 ;
    font-size: 18px ;
}


.footer {
    margin-top: 20px;
    text-align: center;
    color: #f2f2f2;
    font-size: 16px;
    font-weight: 600;
    text-shadow: 1px 1px 2px black;
}



.gradio-tabs {
    background: transparent ;
}


.gradio-tabs .tabitem span {
    color: white ;
}
/* Gradient brown input boxes */
input[type='text'], textarea {
    background: linear-gradient(180deg, rgba(214,180,138,0.25), rgba(180,140,95,0.30)) !important;
    border: 2px solid #b38752 !important;
    border-radius: 12px !important;
    padding: 12px !important;
    font-size: 16px !important;
    color: white !important;
    backdrop-filter: blur(4px);
}



.gradio-tabs .tabitem.selected {
    background: linear-gradient(90deg, #b38752, #84592c) ;
    border-radius: 10px ;
    border-bottom: none ;
}


.gradio-tabs .tabitem:hover {
    background: rgba(255, 255, 255, 0.12) ;
    border-radius: 10px ;
}



.block-label:hover {
    box-shadow: 0 0 10px rgba(214,180,138,0.6);
}

span[class^="svelte-"][data-testid="block-info"] {
    background-color: rgba(214,180,138,0.40) !important;
    color: #4e2a0a !important;
    padding: 6px 12px !important;
    border-radius: 8px !important;
    border: 1px solid #b38752 !important;
    font-weight: 700 !important;
    font-size: 16px !important;
    box-shadow: 0 0 6px rgba(0,0,0,0.25);
}

label.svelte-j0zqjt.float,
label.svelte-j0zqjt,
label[class*="svelte-"].float {
    background-color: rgba(214,180,138,0.40) ;
    color: #4e2a0a !important;
    padding: 6px 12px !important;
    border-radius: 8px !important;
    border: 1px solid #b38752 !important;
    font-weight: 700 !important;
    font-size: 18px ;
    box-shadow: 0 0 6px rgba(0,0,0,0.25);
}



button.svelte-edrmkl.center.boundedheight.flex {
    background: rgba(240, 220, 190, 0.35) ;
    border: 2px solid #b38752 ;
    border-radius: 15px ;
    backdrop-filter: blur(4px);
    padding: 20px ;
}

/* Text inside the upload box */
button.svelte-edrmkl.center.boundedheight.flex * {
    color: #4e2a0a !important;
    font-weight: 600 !important;
}

/* Upload SVG icon (arrow) */
button.svelte-edrmkl.center.boundedheight.flex svg {
    stroke: #4e2a0a !important;
}


button.svelte-i00v67.selected::after {
    background: linear-gradient(90deg, #b38752, #84592c) !important;
    height: 4px !important;  /* adjust thickness */
    bottom: -2px !important;
    border-radius: 2px;
}

label[for*="file-download"] + div {
    background: linear-gradient(180deg, #ecd7bd, #d6b48a) !important;
    border: 2px solid #b38752 !important;
    border-radius: 14px !important;
    padding: 20px !important;
    backdrop-filter: blur(4px);
}


.input-container div,
.input-container .svelte-1ae7ssi,
.input-container *:not(input):not(textarea) {
    color: #5a3b15 !important;
}

.input-container input {
    caret-color: #5a3b15 !important;
}


.typing {
    color: #ffffff !important;
    text-shadow: 0 0 4px rgba(0,0,0,0.6) !important;
}


div.prose.svelte-ydeks8,
div.prose.svelte-ydeks8 * {
    color: #ffffff !important;
    text-shadow: 0 0 4px rgba(0,0,0,0.6) !important;
}


.prose.svelte-ydeks8 h1,
div.prose.svelte-ydeks8 h1,
.prose h1 {
    color: #ffffff !important;
    text-align: center !important;
    font-size: 52px !important;
    font-weight: 900 !important;
    line-height: 1.15 !important;


    text-shadow:
        0 0 6px rgba(255, 210, 160, 0.75),
        0 0 12px rgba(255, 180, 120, 0.55),
        0 0 20px rgba(255, 150, 90, 0.35) !important;

    animation: softGlow 3s ease-in-out infinite alternate !important;
}

@keyframes softGlow {
    from {
        text-shadow:
            0 0 6px rgba(255, 210, 160, 0.75),
            0 0 12px rgba(255, 180, 120, 0.55),
            0 0 20px rgba(255, 150, 90, 0.35);
    }
    to {
        text-shadow:
            0 0 10px rgba(255, 220, 180, 0.9),
            0 0 18px rgba(255, 190, 140, 0.7),
            0 0 28px rgba(255, 160, 100, 0.5);
    }
}



"""



# ------------------ ANIMATED DESCRIPTION ------------------
animated_description = """
<div class="typing" style="color:white; font-size:20px; margin-bottom:15px;">
This AI model predicts the category of any news article instantly...
</div>
"""

footer_html = """
<div class="footer">
Project by <b>Ishleen Kaur</b> (Roll No: <b>80025340021</b>)
</div>
"""


# ------------------ FULL GRADIO APP WITH TABS ------------------
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:

    gr.HTML("<h1>CateX</h1>")
    gr.HTML(animated_description)
    gr.HTML(footer_html)

    with gr.Tabs():

        # ------- TAB 1: MANUAL INPUT -------
        with gr.TabItem("Manual Prediction"):
            title_input = gr.Textbox(label="News Title")
            summary_input = gr.Textbox(label="News Summary", lines=5)
            output_label = gr.Label(label="Predicted Category")
            button = gr.Button("Predict")
            button.click(classify_news, [title_input, summary_input], output_label)

        # ------- TAB 2: CSV UPLOAD -------
        with gr.TabItem("📁 Upload CSV"):
            csv_file = gr.File(label="Upload CSV (must contain Title & Summary)")
            csv_output_table = gr.DataFrame(label="Results Preview")
            csv_download = gr.File(label="Download Result CSV")

            csv_button = gr.Button("Process CSV")
            csv_button.click(
                classify_csv,
                inputs=csv_file,
                outputs=[csv_output_table, csv_download]
            )

demo.launch(debug=True)